#### X Region Migration Script

In [ ]:
%pip install -q semantic-link-labs

In [ ]:
source_ws = 'GovernanceWs' # the source workspace to be migrated
target_ws_suffix = '_XREGION'

target_capacity = 'c1pri'



In [ ]:
import pandas as pd
import datetime
import re,json, fnmatch,os
import requests, base64
import sempy
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException
from pyspark.sql import DataFrame
from pyspark.sql.functions import col,current_timestamp,lit
import sempy_labs as labs
from sempy_labs import migration, directlake
from sempy_labs import lakehouse as lake
from sempy_labs import report as rep
from sempy_labs.tom import connect_semantic_model

# instantiate the Fabric rest client
client = fabric.FabricRestClient()

# get the current workspace ID based on the context of where this notebook is run from
thisWsId = notebookutils.runtime.context['currentWorkspaceId']
thisWsName = notebookutils.runtime.context['currentWorkspaceName']


###### Check the status of the target capacity - it needs to be active to git connect/sync
TODO get region of source workspace capacity and check that the region is different

In [ ]:
# if target capacity name set then obtain the ID. If not set then raise an error 
if target_capacity != '':
    target_capacity_id = labs.resolve_capacity_id(target_capacity)
else:
    raise ValueError(f"Pleas specify a target capacity name in the parameter section")

# get the source workspace capacity details
source_capacity_id = labs.resolve_workspace_capacity(source_ws)[0]
# TODO then get capacity region details from both an check to see they're different


print(f'Target capactiy is set to {target_capacity} ({target_capacity_id}). Checking status...')
dfC = fabric.list_capacities()
dfC_filt = dfC[dfC["Id"] == target_capacity_id]
cap_status = dfC_filt['State'].iloc[0]
if cap_status == 'Inactive':
    raise ValueError(f"Status of capacity {target_capacity} is {cap_status}. Please resume the capacity and retry")
else:
    print(f"Status of capacity {target_capacity} is {cap_status}")


###### Create target workspace

In [ ]:
try:
    target_ws = source_ws + target_ws_suffix
    # create new workspace
    print("Creating workspace: " + target_ws + " in capacity "+ target_capacity_id +"...")
    response = fabric.create_workspace(target_ws,target_capacity_id) 
    new_workspace_id = response
    print("Created workspace with ID: " + new_workspace_id)
except Exception as error:
    errmsg =  "Failed to create workspace " +target_ws + " with capacity ID ("+ target_capacity_id + ") due to: "+str(error)
    print(errmsg)
    #notebookutils.notebook.exit(f'Workspace {target_ws} already exists')
    raise ValueError(f"Workspace {target_ws} already exists. Please use a different workspace name.")



In [ ]:
#Get git connection details for current workspace
gitconnx = labs.get_git_connection(source_ws)

#Connect target workspace
labs.connect_workspace_to_git(
    gitconnx.loc[0, 'Organization Name'],
    gitconnx.loc[0, 'Project Name'],
    gitconnx.loc[0, 'Repository Name'],
    gitconnx.loc[0, 'Branch Name'],
    gitconnx.loc[0, 'Directory Name'],
    gitconnx.loc[0, 'Git Provider Type'],
    target_ws)

# Initialise and sync the items into the target workspace
commit_hash = labs.initialize_git_connection(branch_to_new_ws)
labs.update_from_git(remote_commit_hash=commit_hash,
                     conflict_resolution_policy='PreferWorkspace', 
                     workspace=branch_to_new_ws)

###### Update default lakehouses for notebooks

In [ ]:
for notebook in notebookutils.notebook.list(workspaceId=new_workspace_id):
    if notebook.displayName != 'Create Feature Branch':

        # Get the current notebook definition
        notebook_def = notebookutils.notebook.getDefinition(notebook.displayName,workspaceId=new_workspace_id)
        json_payload = json.loads(notebook_def)
        
        # Check and remove any attached lakehouses
        if 'dependencies' in json_payload['metadata'] and 'lakehouse' in json_payload['metadata']['dependencies']:
        # Remove all lakehouses
            current_lakehouse = json_payload['metadata']['dependencies']['lakehouse']
            json_payload['metadata']['dependencies']['lakehouse'] = {}

            #Update new notebook definition after removing existing lakehouses and with new default lakehouseId
            (notebookutils.notebook.updateDefinition(
                        name = notebook.displayName,
                        content  = json.dumps(json_payload),  
                        defaultLakehouse = current_lakehouse['default_lakehouse_name'],
                        defaultLakehouseWorkspace = new_workspace_id,
                        workspaceId = new_workspace_id
                        )
                )
            print(f"Updated notebook {notebook.displayName} with new default lakehouse: {current_lakehouse['default_lakehouse_name']} in {new_workspace_id}")
       

###### Update direct lake model lakehouse connection

https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.directlake.html#sempy_labs.directlake.update_direct_lake_model_lakehouse_connection
    
TODO: I think MK has a simplified version of this for all direct lake models in the workspace.
TODO: For multi-workspace migration need to adapt this to re-connect to all the migrated workspace lakehouses/warehouses as they may not be in the same workspace 

In [ ]:

df_datasets = fabric.list_datasets(branch_to_new_ws)

# Iterate over each dataset in the dataframe
for index, row in df_datasets.iterrows():
    # Check if the dataset is not the default semantic model
    if not labs.is_default_semantic_model(row['Dataset Name'], fabric.resolve_workspace_id(branch_to_new_ws)):
        print('Updating semantic model connection ' + row['Dataset Name'])
        labs.directlake.update_direct_lake_model_lakehouse_connection(dataset=row['Dataset Name'], workspace= branch_to_new_ws,lakehouse =labs.directlake.get_direct_lake_source(row['Dataset Name'], workspace= branch_to_new_ws)[1], lakehouse_workspace=branch_to_new_ws)
        labs.refresh_semantic_model(dataset=row['Dataset Name'], workspace= branch_to_new_ws)



##### Rebind reports in new branch workspace

Note sure if we still need to do the rebind.

https://semantic-link-labs.readthedocs.io/en/latest/sempy_labs.report.html#sempy_labs.report.report_rebind

In [ ]:
df_reports = fabric.list_reports(workspace=branch_to_new_ws)
for index, row in df_reports.iterrows():
    #print(row['Name'] + '-' + row['Dataset Id'])
    df_datasets = fabric.list_datasets(workspace=branch_to_new_ws)
    dataset_name = df_datasets[df_datasets['Dataset ID'] == row['Dataset Id']]['Dataset Name'].values[0]
    print(dataset_name)
    labs.report.report_rebind(report=row['Name'],dataset=dataset_name, report_workspace=branch_to_new_ws, dataset_workspace=branch_to_new_ws)


##### TODO workspace role assignments and direct shares

I have API code to do this but let's rewrite using SLL

##### TODO update connections e.g. for data pipelines source and sink connections. SLL has functions for this

###### Optional - disconnect from git

In [ ]:
# run this if you want to disconnect from git
#labs.disconnect_workspace_from_git(target_ws)     


#### Code to copy lakehouse and warehouse data

###### Utility functions

In [ ]:
def get_lh_object_list(base_path,data_types = ['Tables', 'Files'])->pd.DataFrame:

    '''
    Function to get a list of tables for a lakehouse
    adapted from https://fabric.guru/getting-a-list-of-folders-and-delta-tables-in-the-fabric-lakehouse
    This function will return a pandas dataframe containing names and abfss paths of each folder for Files and Tables
    '''
    #data_types = ['Tables', 'Files'] #for if you want a list of files and tables
    #data_types = ['Tables'] #for if you want a list of tables

    df = pd.concat([
        pd.DataFrame({
            'name': [item.name for item in notebookutils.fs.ls(f'{base_path}/{data_type}/')],
            'type': data_type[:-1].lower() , 
            'src_path': [item.path for item in notebookutils.fs.ls(f'{base_path}/{data_type}/')],
        }) for data_type in data_types], ignore_index=True)

    return df

def get_wh_object_list(schema_list,base_path)->pd.DataFrame:

    '''
    Function to get a list of tables for a warehouse by schema
    '''
    data_type = 'Tables'
    dfs = []

    for schema_prefix in schema_list:
        if notebookutils.fs.exists(f'{base_path}/{data_type}/{schema_prefix}/'):
            items = notebookutils.fs.ls(f'{base_path}/{data_type}/{schema_prefix}/')
            if items:  # Check if the list is not empty
                df = pd.DataFrame({
                    'schema': schema_prefix,
                    'name': [item.name for item in items],
                    'type': data_type[:-1].lower(),
                    'src_path': [item.path for item in items],
                })
                dfs.append(df)

    if dfs:  # Check if the list of dataframes is not empty
        df = pd.concat(dfs, ignore_index=True)
    else:
        df = pd.DataFrame()  # Return an empty dataframe if no dataframes were created

    return df

def copy_lh_objects(table_list,workspace_src,workspace_tgt,lakehouse_src,lakehouse_tgt,fastcopy=True,usingIDs=False)->pd.DataFrame:
    # declare an array to keep the instrumentation
    cpresult = []
    # loop through all the tables to extract the source path 
    for table in table_list.src_path:
        source = table
        destination = source.replace(f'abfss://{workspace_src}', f'abfss://{workspace_tgt}')
        if usingIDs:
            destination = destination.replace(f'{lakehouse_src}', f'{lakehouse_tgt}')
        else:
            destination = destination.replace(f'{lakehouse_src}.Lakehouse', f'{lakehouse_tgt}.Lakehouse')
        start_time =  datetime.datetime.now()
        if notebookutils.fs.exists(destination):
             notebookutils.fs.rm(destination, True)
        if fastcopy:
            # use fastcopy util which is a python wrapper to azcopy
            notebookutils.fs.fastcp(source+'/*', destination+'/', True)
        else:
            notebookutils.fs.cp(source, destination, True)

        # recording the timing and add it to the results list
        end_time = datetime.datetime.now()
        copyreslist = [source, destination, start_time.strftime("%Y-%m-%d %H:%M:%S"),  end_time.strftime("%Y-%m-%d %H:%M:%S"), str((end_time - start_time).total_seconds())]
        cpresult.append(copyreslist)
    return pd.DataFrame(cpresult,columns =['source--------------------------------------','target--------------------------------------','start------------','end_time------------','elapsed seconds----'])

def createDWrecoverypl(ws_id,pl_name = 'Recover_Warehouse_Data_From_DR'):
  client = fabric.FabricRestClient()

  dfurl= "v1/workspaces/"+ ws_id + "/items"
  payload = { 
  "displayName": pl_name, 
  "type": "DataPipeline", 
  "definition": { 
    "parts": [ 
      { 
        "path": "pipeline-content.json", 
        "payload":  "ewogICAgInByb3BlcnRpZXMiOiB7CiAgICAgICAgImFjdGl2aXRpZXMiOiBbCiAgICAgICAgICAgIHsKICAgICAgICAgICAgICAgICJuYW1lIjogIkl0ZXJhdGVTY2hlbWFUYWJsZXMiLAogICAgICAgICAgICAgICAgInR5cGUiOiAiRm9yRWFjaCIsCiAgICAgICAgICAgICAgICAiZGVwZW5kc09uIjogW10sCiAgICAgICAgICAgICAgICAidHlwZVByb3BlcnRpZXMiOiB7CiAgICAgICAgICAgICAgICAgICAgIml0ZW1zIjogewogICAgICAgICAgICAgICAgICAgICAgICAidmFsdWUiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy50YWJsZXNUb0NvcHkiLAogICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJFeHByZXNzaW9uIgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgImJhdGNoQ291bnQiOiAyMCwKICAgICAgICAgICAgICAgICAgICAiYWN0aXZpdGllcyI6IFsKICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiQ29weVdhcmVob3VzZVRhYmxlcyIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJDb3B5IiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJkZXBlbmRzT24iOiBbCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiYWN0aXZpdHkiOiAiU2V0IHRhYmxlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImRlcGVuZGVuY3lDb25kaXRpb25zIjogWwogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIlN1Y2NlZWRlZCIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgXQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgIF0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAicG9saWN5IjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0aW1lb3V0IjogIjAuMTI6MDA6MDAiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJyZXRyeSI6IDIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInJldHJ5SW50ZXJ2YWxJblNlY29uZHMiOiAzMCwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAic2VjdXJlT3V0cHV0IjogZmFsc2UsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNlY3VyZUlucHV0IjogZmFsc2UKICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZVByb3BlcnRpZXMiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNvdXJjZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRGF0YVdhcmVob3VzZVNvdXJjZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJxdWVyeVRpbWVvdXQiOiAiMDI6MDA6MDAiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAicGFydGl0aW9uT3B0aW9uIjogIk5vbmUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiZGF0YXNldFNldHRpbmdzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFubm90YXRpb25zIjogW10sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAibGlua2VkU2VydmljZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICIwN2EwMzAwNl9kMWI2XzRhMzlfYmViMV8wYmJhMmFhZjVmZjciLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJwcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiYW5ub3RhdGlvbnMiOiBbXSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRGF0YVdhcmVob3VzZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlUHJvcGVydGllcyI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJlbmRwb2ludCI6ICJAcGlwZWxpbmUoKS5wYXJhbWV0ZXJzLmxha2Vob3VzZUNvbm5TdHIiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFydGlmYWN0SWQiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy5sYWtlaG91c2VJZCIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAid29ya3NwYWNlSWQiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy53b3Jrc3BhY2VJZCIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJEYXRhV2FyZWhvdXNlVGFibGUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNjaGVtYSI6IFtdLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGVQcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidGFibGUiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YWx1ZSI6ICJAY29uY2F0KGNvbmNhdChpdGVtKCkuc2NoZW1hLCdfJyksaXRlbSgpLm5hbWUpIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRXhwcmVzc2lvbiIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzaW5rIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJEYXRhV2FyZWhvdXNlU2luayIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJhbGxvd0NvcHlDb21tYW5kIjogdHJ1ZSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInRhYmxlT3B0aW9uIjogImF1dG9DcmVhdGUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiZGF0YXNldFNldHRpbmdzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFubm90YXRpb25zIjogW10sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAibGlua2VkU2VydmljZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICIwYzAzMTIzYV9kMzEyXzQ2YzRfYThlN181YjRjYWQ4ZjEyZDciLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJwcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiYW5ub3RhdGlvbnMiOiBbXSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRGF0YVdhcmVob3VzZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlUHJvcGVydGllcyI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJlbmRwb2ludCI6ICJAcGlwZWxpbmUoKS5wYXJhbWV0ZXJzLndhcmVob3VzZUNvbm5TdHIiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFydGlmYWN0SWQiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy53YXJlaG91c2VJZCIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAid29ya3NwYWNlSWQiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy53b3Jrc3BhY2VJZCIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJEYXRhV2FyZWhvdXNlVGFibGUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNjaGVtYSI6IFtdLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGVQcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidGFibGUiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YWx1ZSI6ICJAaXRlbSgpLm5hbWUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJFeHByZXNzaW9uIgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImVuYWJsZVN0YWdpbmciOiB0cnVlLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0cmFuc2xhdG9yIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJUYWJ1bGFyVHJhbnNsYXRvciIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlQ29udmVyc2lvbiI6IHRydWUsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlQ29udmVyc2lvblNldHRpbmdzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFsbG93RGF0YVRydW5jYXRpb24iOiB0cnVlLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInRyZWF0Qm9vbGVhbkFzTnVtYmVyIjogZmFsc2UKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgfSwKICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiU2V0IHRhYmxlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlIjogIlNldFZhcmlhYmxlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJkZXBlbmRzT24iOiBbCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiYWN0aXZpdHkiOiAiU2V0IHNjaGVtYSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJkZXBlbmRlbmN5Q29uZGl0aW9ucyI6IFsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJTdWNjZWVkZWQiCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIF0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBdLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgInBvbGljeSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAic2VjdXJlT3V0cHV0IjogZmFsc2UsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNlY3VyZUlucHV0IjogZmFsc2UKICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZVByb3BlcnRpZXMiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInZhcmlhYmxlTmFtZSI6ICJUYWJsZW5hbWUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YWx1ZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInZhbHVlIjogIkBpdGVtKCkubmFtZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlIjogIkV4cHJlc3Npb24iCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICJTZXQgc2NoZW1hIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlIjogIlNldFZhcmlhYmxlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJkZXBlbmRzT24iOiBbXSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJwb2xpY3kiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNlY3VyZU91dHB1dCI6IGZhbHNlLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzZWN1cmVJbnB1dCI6IGZhbHNlCiAgICAgICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGVQcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YXJpYWJsZU5hbWUiOiAiU2NoZW1hbmFtZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInZhbHVlIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidmFsdWUiOiAiQGl0ZW0oKS5zY2hlbWEiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJFeHByZXNzaW9uIgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgIF0KICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgfQogICAgICAgIF0sCiAgICAgICAgInBhcmFtZXRlcnMiOiB7CiAgICAgICAgICAgICJsYWtlaG91c2VJZCI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogInN0cmluZyIsCiAgICAgICAgICAgICAgICAiZGVmYXVsdFZhbHVlIjogIjBmMGY2YjdjLTE3NjEtNDFlNi04OTZlLTMwMDE0ZjE2ZmY2ZCIKICAgICAgICAgICAgfSwKICAgICAgICAgICAgInRhYmxlc1RvQ29weSI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogImFycmF5IiwKICAgICAgICAgICAgICAgICJkZWZhdWx0VmFsdWUiOiBbCiAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogImRibyIsCiAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogIkRhdGUiCiAgICAgICAgICAgICAgICAgICAgfSwKICAgICAgICAgICAgICAgICAgICB7CiAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiR2VvZ3JhcGh5IgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogImRibyIsCiAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogIkhhY2tuZXlMaWNlbnNlIgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogImRibyIsCiAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogIk1lZGFsbGlvbiIKICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgIHsKICAgICAgICAgICAgICAgICAgICAgICAgInNjaGVtYSI6ICJkYm8iLAogICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICJUaW1lIgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogImRibyIsCiAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogIlRyaXAiCiAgICAgICAgICAgICAgICAgICAgfSwKICAgICAgICAgICAgICAgICAgICB7CiAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiV2VhdGhlciIKICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICBdCiAgICAgICAgICAgIH0sCiAgICAgICAgICAgICJ3b3Jrc3BhY2VJZCI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogInN0cmluZyIsCiAgICAgICAgICAgICAgICAiZGVmYXVsdFZhbHVlIjogIjE1MDExNDNjLTI3MmYtNGEyZi05NzZhLTdlNTU5NzFlNGMyYiIKICAgICAgICAgICAgfSwKICAgICAgICAgICAgIndhcmVob3VzZUlkIjogewogICAgICAgICAgICAgICAgInR5cGUiOiAic3RyaW5nIiwKICAgICAgICAgICAgICAgICJkZWZhdWx0VmFsdWUiOiAiNGQxYmQ5NTEtOTlkZS00YmQ3LWI3YmMtNzFjOGY1NmRiNDExIgogICAgICAgICAgICB9LAogICAgICAgICAgICAid2FyZWhvdXNlQ29ublN0ciI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogInN0cmluZyIsCiAgICAgICAgICAgICAgICAiZGVmYXVsdFZhbHVlIjogIjcyd3diaXZpMnViZWpicnRtdGFobzMyYjR5LWhxa2FjZmpwZTR4dXZmM2twemt6b2hzbWZtLmRhdGF3YXJlaG91c2UuZmFicmljLm1pY3Jvc29mdC5jb20iCiAgICAgICAgICAgIH0sCiAgICAgICAgICAgICJsYWtlaG91c2VDb25uU3RyIjogewogICAgICAgICAgICAgICAgInR5cGUiOiAic3RyaW5nIiwKICAgICAgICAgICAgICAgICJkZWZhdWx0VmFsdWUiOiAiNzJ3d2JpdmkydWJlamJydG10YWhvMzJiNHktaHFrYWNmanBlNHh1dmYza3B6a3pvaHNtZm0uZGF0YXdhcmVob3VzZS5mYWJyaWMubWljcm9zb2Z0LmNvbSIKICAgICAgICAgICAgfQogICAgICAgIH0sCiAgICAgICAgInZhcmlhYmxlcyI6IHsKICAgICAgICAgICAgIlRhYmxlbmFtZSI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogIlN0cmluZyIKICAgICAgICAgICAgfSwKICAgICAgICAgICAgIlNjaGVtYW5hbWUiOiB7CiAgICAgICAgICAgICAgICAidHlwZSI6ICJTdHJpbmciCiAgICAgICAgICAgIH0KICAgICAgICB9LAogICAgICAgICJsYXN0TW9kaWZpZWRCeU9iamVjdElkIjogIjRhYTIwYWY3LTk0YmQtNDM0OC1iZWY4LWY4Y2JjZDg0MGQ1MSIsCiAgICAgICAgImxhc3RQdWJsaXNoVGltZSI6ICIyMDI0LTExLTEzVDE1OjUyOjUyWiIKICAgIH0KfQ==", 
        "payloadType": "InlineBase64" 
      } 
    ] 
  } 
}   
  
  response = json.loads(client.post(dfurl,json= payload).content)
  return response['id']

def getItemId(wks_id,itm_name,itm_type):
    df = fabric.list_items(type=None,workspace=wks_id)
    #print(df)
    if df.empty:
        return 'NotExists'
    else:
        #display(df)
        #print(df.query('"Display Name"="'+itm_name+'"'))
        if itm_type != '':
            newdf= df.loc[(df['Display Name'] == itm_name) & (df['Type'] == itm_type)]['Id']
        else:
            newdf= df.loc[(df['Display Name'] == itm_name)]['Id']  
        if newdf.empty:
            return 'NotExists'
        else:
            return newdf.iloc[0]


###### Copy lakehouse data

In [ ]:
df_lakehouses = (labs.list_lakehouses(source_ws))
display(df_lakehouses)
for index, row in df_lakehouses.iterrows():
    lh_name= row['Lakehouse Name']
    # Gathers the list of recovers tables and source paths to be copied into the lakehouse associated with this notebook 
    src_path = f'abfss://{source_ws}@onelake.dfs.fabric.microsoft.com/{lh_name}.Lakehouse'

    table_list = get_lh_object_list(src_path)
    print('The following tables will attempt to be recovered and persisted as tables in the default lakehouse of this notebook...')
    display(table_list)

    print('Copy Lakehouse Delta tables...')
    res = copy_lh_objects(table_list[table_list['type']=='table'],source_ws,target_ws,
                        lh_name,lh_name,False,False)
    display(res)
    # Copy files
    print('Copy Lakehouse files...')
    res = copy_lh_objects(table_list[table_list['type']=='file'],source_ws,target_ws,
                        lh_name,lh_name,False,False)
    display(res)
    print('Done.')


###### Copy warehouse data

In [ ]:
p_logging_verbose = True
source_ws_id = fabric.resolve_workspace_id(source_ws)
target_ws_id = fabric.resolve_workspace_id(target_ws)
df_warehouses = (labs.list_warehouses(source_ws))
display(df_warehouses)
for index, row in df_warehouses.iterrows():
    source_wh_id = labs.resolve_warehouse_id(row['Warehouse Name'],source_ws_id)
    target_wh_id = labs.resolve_warehouse_id(row['Warehouse Name'],target_ws_id)
    
    src_path = f'abfss://'+source_ws_id+'@onelake.dfs.fabric.microsoft.com/'+source_wh_id
    tgt_path = f'abfss://'+target_ws_id+'@onelake.dfs.fabric.microsoft.com/'+target_wh_id

    # extract the list of schemas per data 
    schema_list = get_lh_object_list(src_path,['Tables'])
    # extract a list of warehouse objects per schema and store in a list
    table_list = get_wh_object_list(schema_list['name'],src_path)
  
    # create a temporary staging lakehouse per warehouse to create shortcuts into, 
    # which point back to original warehouse data currently in the DR storage account
    lhname = 'temp_rlh_' + source_ws+'_'+row['Warehouse Name']
    # check if it exists before attempting create
    if p_logging_verbose:
        print('Checking whether the temporary lakehouse "'+ lhname +'" exists in workspace '+target_ws+'...')
    temp_lh_id = getItemId(target_ws_id,lhname,'Lakehouse')
    if temp_lh_id == 'NotExists':
        lhname = lhname[:256] # lakehouse name should not exceed 256 characters
        payload = payload = '{"displayName": "' + lhname + '",' \
        + '"description":  "Interim staging lakehouse for primary warehouse recovery: ' \
        + source_ws+'_'+row['Warehouse Name'] + 'into workspace '+ target_ws + '(' + target_ws +')"}'
        try:
            lhurl = "v1/workspaces/" + target_ws_id + "/lakehouses"
            lhresponse = client.post(lhurl,json= json.loads(payload))
            temp_lh_id = lhresponse.json()['id']
            if p_logging_verbose:
                print('Temporary lakehouse "'+ lhname +'" created with Id ' + temp_lh_id + ': ' + str(lhresponse.status_code) + ' ' + str(lhresponse.text))
        except Exception as error:
            print(error.errorCode)
    else:
        if p_logging_verbose:
            print('Temporary lakehouse '+lhname+' (' + temp_lh_id + ') already exists.')
        

    # Create shortcuts for every table in the format of schema_table under the tables folder
    for index,itable in table_list.iterrows():
        shortcutExists=False
        # Check if shortcut exists
        try:
            url = "v1/workspaces/" + target_ws_id + "/items/" + temp_lh_id + "/shortcuts/Tables/"+itable['schema']+'_'+itable['name']
            tlhresponse = client.get(url)
            shortcutExists = True
            if p_logging_verbose:
                print('Shortcut '+itable['schema']+'_'+itable['name'] +' already exists')
        except Exception as error:
            shortcutExists = False    

        if not shortcutExists: 
            # Create shortcuts - one per table per schema
            url = "v1/workspaces/" + target_ws_id + "/items/" + temp_lh_id + "/shortcuts"
            scpayload = '{' \
            '"path": "Tables/",' \
            '"name": "'+itable['schema']+'_'+itable['name']+'",' \
            '"target": {' \
            '"oneLake": {' \
                '"workspaceId": "' + source_ws_id + '",' \
                '"itemId": "'+ source_wh_id +'",' \
                '"path": "/Tables/' + itable['schema']+'/'+itable['name'] + '"' \
                '}}}' 
            try:
                #print(scpayload)                
                shctresponse = client.post(url,json= json.loads(scpayload))
                if p_logging_verbose:
                    print('Shortcut '+itable['schema']+'_'+itable['name'] + ' created.' )

            except Exception as error:
                print('Error creating shortcut '+itable['schema']+'_'+itable['name']+' due to '+str(error) + ':' + shctresponse.text)
    
    recovery_pipeline_prefix= 'plRecover_WH'       
    # recovery pipeline name should not exceed 256 characters
    recovery_pipeline = recovery_pipeline_prefix+'_'+source_ws + '_'+row['Warehouse Name'][:256]
    if p_logging_verbose:
        print('Attempting to deploy a copy pipeline in the target workspace to load the target warehouse tables from the shortcuts created above... ')
    # Create the pipeline in the target workspace that loads the target warehouse from shortcuts created above 
    plid = getItemId( target_ws_id,recovery_pipeline,'DataPipeline')
    #print(plid)
    if plid == 'NotExists':
      plid = createDWrecoverypl(target_ws_id,recovery_pipeline_prefix+'_'+source_ws + '_'+row['Warehouse Name'])
      if p_logging_verbose:
          print('Recovery pipeline ' + recovery_pipeline + ' created with Id '+plid)
    else:
      if p_logging_verbose:
          print('Datawarehouse recovery pipeline "' + recovery_pipeline + '" ('+plid+') already exist in workspace "'+target_ws + '" ('+target_ws_id+')')  
          print('\n')

    tablesToCopyParam = table_list[['schema','name']].to_json( orient='records')
    # ensure the temporary lakehouse exists

    # obtain the connection string for the lakehouse to pass to the copy pipeline
    whurl  = "v1/workspaces/" + target_ws_id + "/lakehouses/" + temp_lh_id
    whresponse = client.get(whurl)
    lhconnStr = whresponse.json()['properties']['sqlEndpointProperties']['connectionString']

    # get the SQLEndpoint ID of the lakehouse to pass to the copy pipeline
    items = fabric.list_items(workspace=target_ws_id)
    temp_lh_sqle_id = items[(items['Type'] == 'SQLEndpoint') & (items['Display Name']==lhname)]['Id'].values[0]


    # obtain the connection string for the warehouse to pass to the copy pipeline    
    whurl  = "v1/workspaces/" + target_ws_id + "/warehouses/" + target_wh_id
    whresponse = client.get(whurl)
    whconnStr = whresponse.json()['properties']['connectionInfo']

    # obtain the pipeline id created to recover this warehouse
    plid = getItemId( i['secondary_ws_id'],recovery_pipeline,'DataPipeline')
    if plid == 'NotExists':
        print('Error: Could not execute pipeline '+recovery_pipeline+ ' as the ID could not be obtained ')
    else:
        # pipeline url including pipeline Id unique to each warehouse
        plurl = 'v1/workspaces/'+target_ws_id+'/items/'+plid+'/jobs/instances?jobType=Pipeline'
        #print(plurl)

        payload_data = '{' \
            '"executionData": {' \
                '"parameters": {' \
                    '"lakehouseId": "' + temp_lh_sqle_id + '",' \
                    '"tablesToCopy": ' + tablesToCopyParam + ',' \
                    '"workspaceId": "' + i['secondary_ws_id'] +'",' \
                    '"warehouseId": "' + i['secondary_id'] + '",' \
                    '"lakehouseConnStr": "' + lhconnStr + '",' \
                    '"warehouseConnStr": "' + whconnStr + '"' \
                    '}}}'
        print(payload_data)
        plresponse = client.post(plurl, json=json.loads(payload_data))
        if p_logging_verbose:
            print(str(plresponse.status_code))      
print('Done')


Note that warehouse recovery requires the intermediate step of copying to a Lakehouse before pipelining into warehouse

In [ ]:
print('Starting warehouse recovery pipelines...')
# interate through all the data warehouses to recover
for idx,i in enumerate(warehousedf):
    if p_logging_verbose:
        print('Invoking pipeline to copy warehouse data from  '+i['primary_ws_name'] + '.'+i['primary_name'] + ' into ' + i['secondary_ws_name'] + '.'+i['secondary_name'] )

    src_path = f'abfss://'+i['primary_ws_id']+'@onelake.dfs.fabric.microsoft.com/'+i['primary_id']
    #tgt_path = f'abfss://'+i['secondary_ws_id']+'@onelake.dfs.fabric.microsoft.com/'+i['secondary_id']

    # extract the list of schemas per data 
    schema_list = get_lh_object_list(src_path,['Tables'])
    #display(schema_list)
    # extract a list of warehouse objects per schema and store in a list
    table_list = get_wh_object_list(schema_list['name'].to_list(),src_path)

    tablesToCopyParam = table_list[['schema','name']].to_json( orient='records')
    # ensure the temporary lakehouse exists
    lhname = 'temp_rlh_' + i['primary_ws_name']+'_'+i['primary_name']
    temp_lh_id = getItemId(i['secondary_ws_id'],lhname,'Lakehouse')
    #temp_lh_id ='0f0f6b7c-1761-41e6-896e-30014f16ff6d'
    
    # obtain the connection string for the lakehouse to pass to the copy pipeline
    whurl  = "v1/workspaces/" + i['secondary_ws_id'] + "/lakehouses/" + temp_lh_id
    whresponse = client.get(whurl)
    lhconnStr = whresponse.json()['properties']['sqlEndpointProperties']['connectionString']

    # get the SQLEndpoint ID of the lakehouse to pass to the copy pipeline
    items = fabric.list_items(workspace=i['secondary_ws_id'])
    temp_lh_sqle_id = items[(items['Type'] == 'SQLEndpoint') & (items['Display Name']==lhname)]['Id'].values[0]


    # obtain the connection string for the warehouse to pass to the copy pipeline    
    whurl  = "v1/workspaces/" + i['secondary_ws_id'] + "/warehouses/" + i['secondary_id']
    whresponse = client.get(whurl)
    whconnStr = whresponse.json()['properties']['connectionInfo']

    recovery_pipeline = recovery_pipeline_prefix+'_'+i['primary_ws_name'] + '_'+i['primary_name'][:256]
    # obtain the pipeline id created to recover this warehouse
    plid = getItemId( i['secondary_ws_id'],recovery_pipeline,'DataPipeline')
    if plid == 'NotExists':
        print('Error: Could not execute pipeline '+recovery_pipeline+ ' as the ID could not be obtained ')
    else:
        # pipeline url including pipeline Id unique to each warehouse
        plurl = 'v1/workspaces/'+i['secondary_ws_id'] +'/items/'+plid+'/jobs/instances?jobType=Pipeline'
        #print(plurl)

        payload_data = '{' \
            '"executionData": {' \
                '"parameters": {' \
                    '"lakehouseId": "' + temp_lh_sqle_id + '",' \
                    '"tablesToCopy": ' + tablesToCopyParam + ',' \
                    '"workspaceId": "' + i['secondary_ws_id'] +'",' \
                    '"warehouseId": "' + i['secondary_id'] + '",' \
                    '"lakehouseConnStr": "' + lhconnStr + '",' \
                    '"warehouseConnStr": "' + whconnStr + '"' \
                    '}}}'
        print(payload_data)
        plresponse = client.post(plurl, json=json.loads(payload_data))
        if p_logging_verbose:
            print(str(plresponse.status_code))      
print('Done')